In [ ]:
## Downloading the Data

In [2]:
import requests
import zipfile
import io

url = "https://archive.ics.uci.edu/static/public/352/online+retail.zip"

r = requests.get(url)
r.raise_for_status()  # check that the download was successful

with zipfile.ZipFile(io.BytesIO(r.content)) as z:
    z.extractall(".")  # extract into current directory

## Loading and Merging Data

In [5]:
import pandas as pd
import numpy as np

# List of possible file paths
file_paths = [
    "Online Retail.xlsx"
]

# Load the first existing file
for path in file_paths:
    try:
        if path.endswith('.xlsx'):
            df_raw = pd.read_excel(path, sheet_name=0)  # read the first sheet
        else:
            df_raw = pd.read_csv(path, low_memory=False)
        print(f"Loaded data from {path}, shape: {df_raw.shape}")
        break
    except FileNotFoundError:
        continue
else:
    raise FileNotFoundError("None of the UCI Online Retail II files were found.")

# Convert InvoiceDate to datetime
df_raw['InvoiceDate'] = pd.to_datetime(df_raw['InvoiceDate'])

# Optional: filter out canceled transactions (InvoiceNo starting with 'C')
df_raw = df_raw[~df_raw['InvoiceNo'].astype(str).str.startswith('C')]

# Create Sales column (equivalent to Rossmann Sales)
df_raw['Sales'] = df_raw['Quantity'] * df_raw['UnitPrice']

print("Preview:")
print(df_raw.head())

Loaded data from Online Retail.xlsx, shape: (541909, 8)
Preview:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  Sales  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  15.30  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  20.34  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  22.00  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  20.34  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  20.34  


In [10]:
# The UCI dataset has InvoiceDate, Quantity, UnitPrice, Customer ID, Country, StockCode, Description
df_raw['InvoiceDate'] = pd.to_datetime(df_raw['InvoiceDate'])
# Normalize column names
df_raw.columns = [c.strip() for c in df_raw.columns]

In [13]:
# Some InvoiceNo start with 'C' indicating cancellations; also remove negative quantities
if 'InvoiceNo' in df_raw.columns:
    df_raw = df_raw[~df_raw['InvoiceNo'].astype(str).str.startswith('C')]

# Filter quantity > 0 and unitprice >= 0
df_raw = df_raw[(df_raw['Quantity'] > 0) & (df_raw['UnitPrice'] >= 0)].copy()

# Create Revenue column
df_raw['Revenue'] = df_raw['Quantity'] * df_raw['UnitPrice']

In [15]:
# Option A: use Country as Store
df_raw['Store'] = df_raw['Country']


# Optional: if UK dominates, you may want to downsample or split UK by top SKUs as pseudo-stores
# Example: create pseudo-stores within UK by assigning top N StockCodes to separate store labels
uk_share = df_raw['Country'].value_counts(normalize=True).get('United Kingdom', 0)
print(f"United Kingdom share: {uk_share:.2%}")

United Kingdom share: 91.53%


In [32]:
# Create 'Date' column at daily frequency
df_raw['Date'] = df_raw['InvoiceDate'].dt.floor('d')

# Create Sales column if not done yet
df_raw['Sales'] = df_raw['Quantity'] * df_raw['UnitPrice']

# Define pseudo-store: either Country or specific high-volume StockCodes
df_raw['Store'] = df_raw['Country']  # simplest approach

# Aggregate daily
daily = df_raw.groupby(['Store', 'Date']).agg(
    Sales=('Sales', 'sum'),
    Transactions=('InvoiceNo', 'nunique'),
    Customers=('CustomerID', lambda x: x.nunique()),  # fixed column name
    Lines=('StockCode', 'count')
).reset_index()

# Fallback if many missing CustomerIDs
if daily['Customers'].isna().mean() > 0.5:
    daily['Customers'] = daily['Transactions']

print('Daily aggregated shape:', daily.shape)
daily.head()

Daily aggregated shape: (1554, 6)


,Store,Date,Sales,Transactions,Customers,Lines
0,Australia,2010-12-01,358.25,1,1,14
1,Australia,2010-12-08,258.90,1,1,8
2,Australia,2010-12-17,415.70,1,1,10
3,Australia,2011-01-06,7154.38,2,2,48
4,Australia,2011-01-10,81.60,1,1,1


In [36]:
# Open flag
daily['Open'] = (daily['Sales'] > 0).astype(int)

# Compute daily avg unit price per Store
price_df = (
    df_raw.groupby(['Store', 'Date'], as_index=False)
    .apply(lambda g: pd.Series({
        'DailyAvgPrice': g['Revenue'].sum() / g['Quantity'].sum() if g['Quantity'].sum() > 0 else np.nan
    }))
)

# Merge to daily
daily = daily.merge(price_df, on=['Store', 'Date'], how='left')

# Compute rolling 30-day median per store efficiently
daily.sort_values(['Store', 'Date'], inplace=True)
daily['Price30dMedian'] = (
    daily.groupby('Store')['DailyAvgPrice']
    .apply(lambda x: x.shift(1).rolling(window=30, min_periods=7).median())
    .reset_index(level=0, drop=True)
)

# Promo flag
daily['Promo'] = ((daily['DailyAvgPrice'] < 0.95 * daily['Price30dMedian']) & daily['DailyAvgPrice'].notna()).astype(int)

# Fill NaNs safely
daily['DailyAvgPrice'] = daily['DailyAvgPrice'].fillna(daily['DailyAvgPrice'].median())

/var/folders/zs/dpjs6jdx52xgnf97mcsn19240000gp/T/ipykernel_44047/1242461860.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [37]:
daily['Year'] = daily['Date'].dt.year
daily['Month'] = daily['Date'].dt.month
daily['Day'] = daily['Date'].dt.day
# ISO week
daily['WeekOfYear'] = daily['Date'].dt.isocalendar().week.astype(int)
# Monday=0
daily['DayOfWeek'] = daily['Date'].dt.dayofweek

In [40]:
from workalendar.europe import UnitedKingdom

# We'll flag UK holidays for the United Kingdom store; for other countries, this block can be extended
cal = UnitedKingdom()

# Create IsHoliday default 0
daily['IsHoliday'] = 0

uk_mask = daily['Store'] == 'United Kingdom'
if uk_mask.any():
    uk_dates = daily.loc[uk_mask, 'Date'].dt.date.unique()
    uk_holidays = set(d for d, _ in cal.holidays(2009) + cal.holidays(2010) + cal.holidays(2011))
    # Workalendar returns list of tuples, so above is illustrative; safer to generate per-year
    holidays_all = set()
    for y in daily['Year'].unique():
        try:
            holidays_all.update([d for d, _ in cal.holidays(y)])
        except Exception:
            pass
    daily.loc[uk_mask, 'IsHoliday'] = daily.loc[uk_mask, 'Date'].dt.date.isin(holidays_all).astype(int)

In [42]:
# Create store-level metadata using historical activity
store_meta = df_raw.groupby('Store').agg(
    NumUniqueSKUs=('StockCode', 'nunique'),
    TotalRevenue=('Revenue', 'sum'),
    FirstSale=('Date', 'min')
).reset_index()

# Assortment: bin NumUniqueSKUs
store_meta['Assortment'] = pd.qcut(store_meta['NumUniqueSKUs'].rank(method='first'), q=3, labels=['a', 'b', 'c'])

# StoreType: map by region heuristic from country name (simple)
def map_store_type(country):
    eu = ['United Kingdom', 'France', 'Germany', 'Spain', 'Netherlands', 'Belgium', 'Ireland', 'Portugal', 'Italy']
    if country in eu:
        return 'a'
    if 'United' in country or 'Kingdom' in country:
        return 'a'
    if country == 'EIRE' or country == 'Ireland':
        return 'b'
    return 'c'

store_meta['StoreType'] = store_meta['Store'].apply(map_store_type)

# CompetitionDistance synthetic: use inverse of TotalRevenue as "distance" (smaller revenue -> farther)
max_rev = store_meta['TotalRevenue'].max()
store_meta['CompetitionDistance'] = (max_rev - store_meta['TotalRevenue']) / max_rev * 100000

# Merge store_meta into daily
daily = daily.merge(store_meta[['Store', 'StoreType', 'Assortment', 'CompetitionDistance']], on='Store', how='left')

In [43]:
# Sort first
daily.sort_values(['Store', 'Date'], inplace=True)

lags = [1, 7]
for lag in lags:
    daily[f'Sales_lag_{lag}'] = daily.groupby('Store')['Sales'].shift(lag)

windows = [7, 28]
for window in windows:
    daily[f'Sales_rolling_mean_{window}'] = daily.groupby('Store')['Sales'].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).mean())
    daily[f'Sales_rolling_std_{window}'] = daily.groupby('Store')['Sales'].transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).std())

# Fill NaNs conservatively: forward fill within store then global median
daily.groupby('Store').apply(lambda g: g.fillna(method='bfill').fillna(method='ffill'))

# After groupwise fills, remaining NaNs -> fill with global median or 0
num_cols = daily.select_dtypes(include=[np.number]).columns.tolist()
daily[num_cols] = daily[num_cols].fillna(daily[num_cols].median())

/var/folders/zs/dpjs6jdx52xgnf97mcsn19240000gp/T/ipykernel_44047/2623317427.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  daily.groupby('Store').apply(lambda g: g.fillna(method='bfill').fillna(method='ffill'))
/var/folders/zs/dpjs6jdx52xgnf97mcsn19240000gp/T/ipykernel_44047/2623317427.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  daily.groupby('Store').apply(lambda g: g.fillna(method='bfill').fillna(method='ffill'))


In [44]:
def encode_cyclical(df, col, max_val):
    df[col + '_sin'] = np.sin(2 * np.pi * df[col]/max_val)
    df[col + '_cos'] = np.cos(2 * np.pi * df[col]/max_val)
    return df

daily = encode_cyclical(daily, 'DayOfWeek', 7)
daily = encode_cyclical(daily, 'Month', 12)
daily = encode_cyclical(daily, 'Day', 31)
daily = encode_cyclical(daily, 'WeekOfYear', 52)

In [45]:
# Competition duration (synthetic) - example: use FirstSale as proxy for competitor absent/present
# For UCI we don't have CompetitionOpenSince — create zeros
daily['CompetitionDurationMonths'] = 0.0

# Promo on weekend interaction
daily['IsWeekend'] = daily['DayOfWeek'].isin([5,6]).astype(int)
daily['PromoOnWeekend'] = daily['Promo'] * daily['IsWeekend']

In [46]:
cols_to_drop = ['DailyAvgPrice', 'Price30dMedian']
daily.drop(columns=[c for c in cols_to_drop if c in daily.columns], inplace=True, errors='ignore')

# Save processed parquet
out_fp = 'processed_train_uci.parquet'
daily.to_parquet(out_fp, index=False)
print('Saved processed data to', out_fp)

# Also save store metadata for teaching merge examples
store_meta.to_csv('country_metadata.csv', index=False)
print('Saved synthetic store metadata to country_metadata.csv')

Saved processed data to processed_train_uci.parquet
Saved synthetic store metadata to country_metadata.csv
